In [ ]:
%load_ext blackcellmagic
%alias_magic -c -p "-l 120" b black

In [ ]:
import os

import datajoint as dj

dj.config["database.host"] = "experimental.db"
dj.config["database.user"] = "alice"
dj.config["database.password"] = "alice-pw"

dj.config["stores"] = dict(
    experimental_minio=dict(
        protocol="s3",
        endpoint="experimental.minio:9000",
        bucket="experimental-bucket",
        location="dj-store",
        access_key="experimental",
        secret_key="experimental_secret_key",
    )
)

schema = dj.schema("alice_main")

In [ ]:
schema.drop()

In [ ]:
n_entities = 10
file_size = 1024
temp_dir = "temp"

In [ ]:
@schema
class Experiment(dj.Manual):
    definition = """
    p_a: int
    ---
    s_a: int
    s_b: attach@experimental_minio # attach@experimental
    """

    class Electrode(dj.Part):
        definition = """
        -> master
        p_b: int
        ---
        s_c: int
        """

    class Screen(dj.Part):
        definition = """
        -> master
        ---
        s_d: attach@experimental_minio
        s_e: int
        """

In [ ]:
def create_random_files(name, n):
    if not os.path.exists(temp_dir):
        os.mkdir(temp_dir)
    file_paths = []
    for i in range(n):
        file_path = os.path.join(temp_dir, name + str(i) + ".rand")
        with open(file_path, "wb") as f:
            f.write(os.urandom(file_size))
        file_paths.append(file_path)
    return file_paths


def create_entities(names, n):
    for name, kind in names.items():
        if isinstance(kind, list):
            continue
        names[name] = list(range(n))
    return [{n: x for n, x in zip(names, d)} for d in zip(*names.values())]


def remove_files(file_paths):
    for file_path in file_paths:
        os.remove(file_path)

In [ ]:
file_paths = create_random_files("experiment", n_entities)
entities = create_entities(dict(p_a=None, s_a=None, s_b=file_paths), n_entities)
Experiment().insert(entities, skip_duplicates=True)
remove_files(file_paths)

In [ ]:
Experiment().Electrode().insert([dict(p_a=x, p_b=x, s_c=x) for x in range(10)], skip_duplicates=True)

In [ ]:
file_paths = create_random_files("screen", n_entities)
entities = create_entities(dict(p_a=None, s_d=file_paths, s_e=None), n_entities)
Experiment().Screen().insert(entities, skip_duplicates=True)
remove_files(file_paths)

In [ ]:
print(Experiment())
print(Experiment().Electrode())
print(Experiment().Screen())

In [ ]:
(Experiment() & "p_a = 3").delete()